### 1. "Телышев" - Вариант №2

In [1]:
surname = "Телышев"  # Ваша фамилия

alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
variants = [4, 42, 21, 21, 34,  1, 44, 26, 18, 43, 38, 26, 18, 43,  3, 49, 45,
            7, 42, 25,  4,  9, 36, 33, 31, 29,  5, 31,  4, 19, 24, 27, 33]
d = dict(zip(alphabet, variants))
variant =  sum([d[element] for element in surname.lower()]) % 2 + 1
print("Ваш вариант - ", variant)

Ваш вариант -  2


### 2. Рейтинги

#### Задача 1. Средний рейтинг
```
Реализуйте подсчет среднего рейтинга продуктов. Результат сохранить в HDFS в файле "avg_rating.csv".
Формат каждой записи: ProdId,Rating
```

In [2]:
rdd_products = sc.textFile('/user/cloudera/hw_part_1/spark/task_1/data/samples_100.json')
# rdd_products = sc.textFile('/user/cloudera/hw_part_1/spark/task_1/data/Electronics.json')

In [5]:
import json

def get_product_rating(product_json):
    product = json.loads(product_json)
    return (product.get('asin'), product.get('overall'))

rdd_product_rating = rdd_products.map(lambda product_json: get_product_rating(product_json))
rdd_product_rating.take(10)

[('0528881469', 5.0),
 ('0528881469', 1.0),
 ('0528881469', 3.0),
 ('0528881469', 2.0),
 ('0528881469', 1.0),
 ('0594451647', 5.0),
 ('0594451647', 2.0),
 ('0594451647', 5.0),
 ('0594451647', 4.0),
 ('0594451647', 5.0)]

In [15]:
rdd_product_avg_rating = (
    rdd_product_rating
        .aggregateByKey(
                (0, 0),
                lambda sum_count, rating: (sum_count[0] + rating, sum_count[1] + 1),
                lambda sum_count_x, sum_count_y: (sum_count_x[0] + sum_count_y[0],
                                                  sum_count_x[1] + sum_count_y[1])
            )
        .mapValues(lambda sum_count: sum_count[0] / sum_count[1])
)
print(rdd_product_avg_rating.collect())

def to_csv_line(data):
    delimiter = ','
    fields = []
    for item in data:
        field = str(item)
        if delimiter in field:
            field = '"{0}"'.format(field)
        fields.append(field)
    return delimiter.join(fields)

print(rdd_product_avg_rating.map(to_csv_line).collect())

[('0972683275', 4.390243902439025), ('0528881469', 2.4), ('0594451647', 4.2), ('0594481813', 4.0)]
['0972683275,4.390243902439025', '0528881469,2.4', '0594451647,4.2', '0594481813,4.0']


```python
# product_avg_rating.py
import json
import sys
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('ProductAvgRating')
sc = SparkContext(conf=conf)

rdd_products = sc.textFile(sys.argv[1])

def get_product_rating(product_json):
    product = json.loads(product_json)
    return (product.get('asin'), product.get('overall'))

rdd_product_rating = rdd_products.map(lambda product_json: get_product_rating(product_json))
rdd_product_avg_rating = (
    rdd_product_rating
        .aggregateByKey(
                (0, 0),
                lambda sum_count, rating: (sum_count[0] + rating, sum_count[1] + 1),
                lambda sum_count_x, sum_count_y: (sum_count_x[0] + sum_count_y[0],
                                                  sum_count_x[1] + sum_count_y[1])
            )
        .mapValues(lambda sum_count: sum_count[0] / sum_count[1])
)

def to_csv_line(data):
    delimiter = ','
    fields = []
    for item in data:
        field = str(item)
        if delimiter in field:
            field = '"{0}"'.format(field)
        fields.append(field)
    return delimiter.join(fields)

rdd_product_avg_rating.map(to_csv_line).saveAsTextFile(sys.argv[2])
```

#### Задача 2. Добавление наименования продукта
```
Напишите программу, которая каждому ProdId из "avg_rating.csv" ставит в соответстие названием продукта.
Результат сохранить в HDFS в файле "prodname_avg_rating.csv": ProdId,Name,Rating
```

In [ ]:
hdfs dfs -rm -r -f -skipTrash /user/cloudera/hw_part_1/spark/task_2
hdfs dfs -mkdir -p /user/cloudera/hw_part_1/spark/task_2/data

hdfs dfs -copyFromLocal /home/cloudera/Desktop/data_samples/avg_rating.csv \
                        /user/cloudera/hw_part_1/spark/task_2/data/avg_rating.csv

hdfs dfs -copyFromLocal /home/cloudera/Desktop/data_samples/samples_100_meta.json \
                        /user/cloudera/hw_part_1/spark/task_2/data/samples_100_meta.json

# hdfs dfs -copyFromLocal /home/cloudera/Desktop/data_samples/Electronics_meta.json \
#                         /user/cloudera/hw_part_1/spark/task_2/data/Electronics_meta.json

hdfs dfs -ls -R /user/cloudera/hw_part_1/spark/task_2/

In [11]:
rdd_products = sc.textFile('/user/cloudera/hw_part_1/spark/task_2/data/avg_rating.csv')
#rdd_products_meta = sc.textFile('/user/cloudera/hw_part_1/spark/task_2/data/samples_100_meta.json')
rdd_products_meta = sc.textFile('/user/cloudera/hw_part_1/spark/task_2/data/Electronics_meta.json')

In [29]:
rdd_product_avg_rating = rdd_products.map(lambda csv_line: csv_line.split(','))
rdd_product_avg_rating.persist()
rdd_product_avg_rating.take(5)

[['B00ARSNT7Q', '4.29411764706'],
 ['B001UHMVC2', '4.55555555556'],
 ['B001WMFXJ8', '4.5'],
 ['B002YNY8GI', '4.46511627907'],
 ['B008JH59FC', '4.22222222222']]

In [30]:
import ast
import json

def get_product_name(product_json):
    # Hack for "JSONDecodeError: Expecting property name enclosed in double quotes"
    product = ast.literal_eval(product_json)
    return (product.get('asin'), product.get('title'))

rdd_product_name = (
    rdd_products_meta
        .map(lambda product_json: get_product_name(product_json))
        .filter(lambda product_data: all(product_data))
)
rdd_product_name.persist()
rdd_product_name.take(5)

[('0132793040',
  'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'),
 ('0321732944',
  'Kelby Training DVD: Adobe Photoshop CS5 Crash Course By Matt Kloskowski'),
 ('0439886341', 'Digital Organizer and Messenger'),
 ('0511189877', 'CLIKR-5 Time Warner Cable Remote Control UR5U-8780L'),
 ('0528881469',
  'Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS')]

In [37]:
rdd_product_name_rating = rdd_product_avg_rating.join(rdd_product_name)
# print(rdd_product_name_rating.take(5))
# [('B008JH59FC', ('4.22222222222', 'Logitech 910-002951 M325 Wireless Mouse - Coral Fan')),]

def to_csv_line(data):
    delimiter = ','
    fields = []
    for item in data:
        field = str(item)
        if delimiter in field:
            field = '"{0}"'.format(field)
        fields.append(field)
    return delimiter.join(fields)

print((
    rdd_product_name_rating
        .map(lambda id__rating_name: (id__rating_name[0], id__rating_name[1][1], id__rating_name[1][0]))
        .map(to_csv_line)
        .take(5)
))


['B008JH59FC,4.22222222222,Logitech 910-002951 M325 Wireless Mouse - Coral Fan', 'B00C5R8A6W,4.66666666667,"Leef Spark USB 2.0 32GB High-speed USB Flash Drive with Magnet Cap, LED, and PrimeGrade Memory (Black)"', 'B0083E7X64,2.83333333333,"HHI Anti-Fingerprint, Anti-Glare, Matte Finished Screen Protector for Toshiba Excite 10 (Package include a HandHelditems Sketch Stylus Pen)"', 'B003OBUJGW,3.66666666667,Sony ECMSST1 Compact Stereo Microphone for NEX-3/NEX-5 Camera', 'B004LPW2LA,3.71428571429,SANOXY&reg; high speed Mini USB Wireless Lan Adaptor 802.11n']


```python
# product_name_rating.py
import ast
import json
import sys
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('ProductAvgRating')
sc = SparkContext(conf=conf)

rdd_products = sc.textFile(sys.argv[1])
rdd_products_meta = sc.textFile(sys.argv[2])

rdd_product_avg_rating = rdd_products.map(lambda csv_line: csv_line.split(','))
rdd_product_avg_rating.persist()

def get_product_name(product_json):
    # Hack for "JSONDecodeError: Expecting property name enclosed in double quotes"
    product = ast.literal_eval(product_json)
    return (product.get('asin'), product.get('title'))

rdd_product_name = (
    rdd_products_meta
        .map(lambda product_json: get_product_name(product_json))
        .filter(lambda product_data: all(product_data))
)
rdd_product_name.persist()

def to_csv_line(data):
    delimiter = ','
    fields = []
    for item in data:
        field = str(item)
        if delimiter in field:
            field = '"{0}"'.format(field)
        fields.append(field)
    return delimiter.join(fields)

rdd_product_name_rating = rdd_product_avg_rating.join(rdd_product_name)
(
    rdd_product_name_rating
        .map(lambda id__rating_name: (id__rating_name[0], id__rating_name[1][1], id__rating_name[1][0]))
        .map(to_csv_line)
        .saveAsTextFile(sys.argv[3])
)
```

In [ ]:
B001CNR39U,OCZ OCZ3M10664GK PC3-8500 DDR3 1066MHz SODIMM 4 GB Dual Channel Kit,4.2
B00419SI1E,"Laptop Notebook 15-Inch 15.6-Inch Skin Sticker Cover Art Decal Fits Laptop Size of 13-Inch,13.3-Inch,14-Inch,15-Inch,15.6-Inch,16-Inch HP Dell Lenovo Asus Compaq Asus Acer Computers",4.0
B002AKHHHM,Discovery Expedition Entertainment Projector,2.5
B0044D3R7M,"Infinity Primus P143 Two-way 4-Inch Bookshelf/Satellite Speaker (Black, Each)",4.875
B000687QW6,Kenwood KFC-415C 160-Watt 4-Inch x 10-Inch Two-Way Speaker System,4.625
B0081HD3PI,On Networks N150 WiFi Router (N150R),3.69230769231
B003MAZY6A,Your Cable Store Black 15 Foot USB 2.0 High Speed Male A To Male A Cable,4.68
B0013T2T6U,Cell Phone/PDA Traveler's Adapter with USB,3.85714285714
B0018O0ZTS,Manfrotto 030- 14 Replacement Hexagonal Quick Release Plate with 1/4- 20 Thread,4.83333333333
B000RXYV5U,IronKey 4 GB Secure Hardware-Encrypted USB 2.0 Flash Drive D20402A,4.5